In [1]:
#takes string of integer (0-9) as input and outputs that integer superscripted as a string
def Superscript(script):
    string = ""
    unicodeList = ["\u2070", "\u00B9", "\u00B2", "\u00B3", "\u2074", "\u2075", "\u2076", "\u2077", "\u2078", "\u2079"]
    for char in script:
        string += unicodeList[int(char)]
    return(string)

#takes string of integer as input (0-9) and outputs that integer subscripted
def Subscript(script):
    string = ""
    unicodeList = ["\u2080", "\u2081", "\u2082", "\u2083", "\u2084", "\u2085", "\u2086", "\u2087", "\u2088", "\u2089"]
    for char in script:
        string += unicodeList[int(char)]
    return(string)

#prompt the user to enter a polynomial in the following term.
print("Enter a polynomial in this form x_i^a * x_j^b + ...")
polynomial = input("W = ")

#clean up the polynomial so that it is just the variable index and exponent
#take these values and create an exponent matrix
cleanPolynomial = polynomial.replace("x_", "")
cleanPolynomial = cleanPolynomial.replace(" ", "")
monomials = cleanPolynomial.split('+')
exponentMatrix = []
for i in range(len(monomials)):
    row = []
    for j in range(len(monomials)):
        row.append(0)
    exponentMatrix.append(row)
for i in range(len(monomials)):
    variables = monomials[i].split('*')
    for j in range(len(variables)):
        index = int(variables[j].split('^')[0])
        exponent = int(variables[j].split('^')[1])
        exponentMatrix[i][index] = exponent

#get the number of variables and convert the list of lists into a matrix as defined by sage
numVariables = len(exponentMatrix)
sageMatrix = matrix(exponentMatrix)
transposeSageMatrix = sageMatrix.transpose()

#print the W along with its corresponding exponent matrix
print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Printing the polynomials and inverse exponent matricies~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
polynomialClean = "\nW = "
for i in range(numVariables):
    for j in range(numVariables):
        if (sageMatrix[i][j] != 0):
            polynomialClean += "x{}{}".format(Subscript(str(j)),Superscript(str(sageMatrix[i][j])))
    if (i < numVariables - 1):
        polynomialClean += " + "
print(polynomialClean)
AInverse = sageMatrix.inverse()
print("A_W^(-1):")
print(AInverse) 

#print the W^T along with its corresponding exponent matrix
transposePoly = "\nW^T = "
for i in range(numVariables):
    for j in range(numVariables):
        if (transposeSageMatrix[i][j] != 0):
            transposePoly += "x{}{}".format(Subscript(str(j)),Superscript(str(transposeSageMatrix[i][j])))
    if (i < numVariables - 1):
        transposePoly += " + "
print(transposePoly)
ATInverse = transposeSageMatrix.inverse()
print("A_(W^T)^(-1):")
print(ATInverse) 

Enter a polynomial in this form x_i^a * x_j^b + ...
W = x_0^5 + x_1^5 + x_2^5 + x_3^5 + x_4^5

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Printing the polynomials and inverse exponent matricies~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

W = x₀⁵ + x₁⁵ + x₂⁵ + x₃⁵ + x₄⁵
A_W^(-1):
[1/5   0   0   0   0]
[  0 1/5   0   0   0]
[  0   0 1/5   0   0]
[  0   0   0 1/5   0]
[  0   0   0   0 1/5]

W^T = x₀⁵ + x₁⁵ + x₂⁵ + x₃⁵ + x₄⁵
A_(W^T)^(-1):
[1/5   0   0   0   0]
[  0 1/5   0   0   0]
[  0   0 1/5   0   0]
[  0   0   0 1/5   0]
[  0   0   0   0 1/5]


In [2]:
#this module allows us to take the cartesian product
from itertools import product

#explanation printed to the user
print("This program will generate a group from a set of generators.")
print("Note: this generator takes values as roots of unity. e.g. 1/3 will be interpreted as the thrid root of unity.")
print("Entering \'0\' or \'1\' will give you exactly 0 or 1 respectively.\n")

#we prompt the user through inputting the generators of the group as matricies
command = ""
firstGenerator = True
generators = []
generatorOrders = []
while command != "n":
    #prompt nothing on the first generator
    if firstGenerator:
        command = ""
        firstGenerator = False
    #ask the user if they wish to continue inputting generators
    else:
        command = input("\nAre there more generators? (y/n) ")
    #if the user doesn't say 'no' to more generators
    if (command != "n"):
        matrixSize = numVariables
        matrixRows = []
        print("Please input each row of the matrix (size {}) in additive notation written as a fraction seperated by commas and no spaces.".format(numVariables))
        rowNum = 0;
        #run through each row and prompt the user to input each element
        while rowNum < matrixSize:
            rowSTR = input("Row " + str(rowNum+1) + ": ")
            rowList = rowSTR.split(',')
            #check if the row inputted meets the size requirement for the polynomail given before
            if (len(rowList) != numVariables):
                print("This row has too few elements! Please try again")
                rowNum -= 1
            else:
                for j in range(len(rowList)):
                    if (rowList[j] == '0'):
                        rowList[j] = 0
                    elif (rowList[j] == '1'):
                        rowList[j] = 1
                    else:
                        numerator = int(rowList[j].split('/')[0])
                        denominator = int(rowList[j].split('/')[1])
                        rowList[j] = e^(2*pi*I*numerator/denominator)
                matrixRows.append(rowList)
            rowNum += 1
        
        #verify with the user if the inputted matrix is correct
        verifyMatrix = matrix(CDF, matrixSize, (matrixRows))
        print("The matrix you just inputed is:")
        print(verifyMatrix)
        verify = input("Is the above matrix correct? (y/n) ")
        #if the matrix is correct append it to the list of generators
        if (verify == "y"):
            elementMatrix = verifyMatrix
            generators.append(elementMatrix)
            
            #find the order of the matrix and append it to the generator's orders
            orderFound = False
            order = 0
            while not orderFound:
                order += 1
                #check if the matrix rasied to the order is within a certain tolerance to the identity matrix
                if (abs((elementMatrix^order - identity_matrix(numVariables)).norm('frob')) < 0.01): 
                    orderFound = True
            elementOrder = order
            print("The calculated order of this element is", str(elementOrder))
            generatorOrders.append(elementOrder)
        #don't save the matrix if it wasn't inputted correctly
        else:
            print("This matrix was removed")
        
#create the list of cartesian products for the powers possible for the generating the group
#this outputs a list of integers that are used as powers on the generators
iterableList = []
for i in range(len(generators)):
    elementOrder = generatorOrders[i]
    iterableSet = [*range(elementOrder)]
    iterableList.append(iterableSet)
cartesianProduct = list(product(*iterableList))

#use the cartesian product elements to calculate the group elements
#save both the actual matrix and the generator products representation
groupElements = []
groupElementsSTR = []
for elementPowers in cartesianProduct:
    elementSTR = ""
    elementMatrix = identity_matrix(numVariables)
    for index, power in enumerate(elementPowers):
        elementSTR += "g{}{} ".format(Subscript(str(index)), Superscript(str(power)))
        elementMatrix *= generators[index]^power
    groupElementsSTR.append(elementSTR)
    groupElements.append(elementMatrix)

#print the group elements
print("\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Printing the group~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("This group has order", str(len(groupElements)))
for index, element in enumerate(groupElementsSTR):
    print("\nGroup Element:", str(index))
    print(element)

This program will generate a group from a set of generators.
Note: this generator takes values as roots of unity. e.g. 1/3 will be interpreted as the thrid root of unity.
Entering '0' or '1' will give you exactly 0 or 1 respectively.

Please input each row of the matrix (size 5) in additive notation written as a fraction seperated by commas and no spaces.
Row 1: 0,1,0,0,0
Row 2: 0,0,1,0,0
Row 3: 0,0,0,1,0
Row 4: 0,0,0,0,1
Row 5: 1,0,0,0,0
The matrix you just inputed is:
[0.0 1.0 0.0 0.0 0.0]
[0.0 0.0 1.0 0.0 0.0]
[0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 1.0]
[1.0 0.0 0.0 0.0 0.0]
Is the above matrix correct? (y/n) y
The calculated order of this element is 5

Are there more generators? (y/n) y
Please input each row of the matrix (size 5) in additive notation written as a fraction seperated by commas and no spaces.
Row 1: 0,0,0,1,0
Row 2: 0,0,1,0,0
Row 3: 0,1,0,0,0
Row 4: 1,0,0,0,0
Row 5: 0,0,0,0,1
The matrix you just inputed is:
[0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 1.0 0.0 0.0]
[0.0 1.0 0.0 0.0

In [3]:
#get the conjugacy classes
print("~~~~~~~Finding Conjugacy Classes~~~~~~~")

#create a copy of the group elements
tmpGE = groupElements.copy()
groupConjugacyClasses = []
paddingNum = len(str(len(tmpGE)))
#continue until the temporary group has no elements
while len(tmpGE) != 0:
    #this is for the progress notification
    paddedGroupLength = str(len(tmpGE)).zfill(paddingNum) 
    print("\rLength of tmpGE:", paddedGroupLength, end = '\r', flush = True)
    
    #take the first element of the temporary group elements and calculate its conjugates
    conjugates = []
    for j in range(len(groupElements)):
        conjugate = groupElements[j] * tmpGE[0] * groupElements[j].inverse()
        
        #check the conjugate against the elements in the group and find the one it is closets to
        #this must be done due to rounding error in calculating the conjugates
        winnerIndex = 0
        winnerScore = 1000
        for i in range(len(tmpGE)):
            score = (conjugate - tmpGE[i]).norm('frob')
            if (winnerScore > abs(score)):
                winnerScore = score
                winnerIndex = i
        
        #only append the conjugates that haven't appeared yet for this class
        conjugate = tmpGE[winnerIndex]
        if conjugate not in conjugates:
            conjugates.append(conjugate)  

    #remove the elements of conjugacy class from the temporary group since conjugacy classes partitions the set
    #we don't need to check these elements again
    for conjugate in conjugates:
        tmpGE.remove(conjugate)
    groupConjugacyClasses.append(conjugates)

#this sets the progress to 0 when finished
print("\rLength of tmpGE:", str(0).zfill(paddingNum), end = '\r', flush = True)
print("\nThe number of conjugacy classes are", len(groupConjugacyClasses), "\n")

#a quick double check that the number of elements in all of the conjugacy classes is equal to
#the number of elements in the group
print("Double check")
sizeCheck = 0
for conjugacyClass in groupConjugacyClasses:
    sizeCheck += len(conjugacyClass)
print("The number of elements in all of the conjugacy classes should be " + str(len(groupElements)))
print("The code has calculated", sizeCheck)

# #print off the conjugacy classes    
# print("\n~~~~~~~~~~~~~~~~~~~~")
# for index, conjugacyClass in enumerate(groupConjugacyClasses):
#     print("Conjugacy Class number: " + str(index))
#     print("The size of this conjugacy class is", len(conjugacyClass), "\n")
#     for element in conjugacyClass:
#         print(element, "\n")
#     print("~~~~~~~~~~~~~~~~~~~~")

~~~~~~~Finding Conjugacy Classes~~~~~~~
Length of tmpGE: 00
The number of conjugacy classes are 4 

Double check
The number of elements in all of the conjugacy classes should be 10
The code has calculated 10


In [4]:
#get the index of every element in all conjugacy class within the group and save another copy
#of conjugacy classes for the group product representation
groupConjugacyClassesSTR = []
for index, conjugacyClass in enumerate(groupConjugacyClasses):
    print("\rWorking on conjugacy class:", index+1, "out of", len(groupConjugacyClasses), end = '\r', flush = True)
    conjugacyClassSTR = []
    for conjugate in conjugacyClass:
        for index, element in enumerate(groupElements):
            if (element == conjugate):
                conjugacyClassSTR.append(groupElementsSTR[index])
    groupConjugacyClassesSTR.append(conjugacyClassSTR)

#print the conjugacy classes in terms of the group product representation
for index, conjugacyClassSTR in enumerate(groupConjugacyClassesSTR):
    print("Conjugacy Class number: " + str(index))
    print("The size of this conjugacy class is", len(conjugacyClassSTR), "\n")
    for element in conjugacyClassSTR:
        print(element, "\n")
    print("~~~~~~~~~~~~~~~~~~~~")

Conjugacy Class number: 0s: 4 out of 4
The size of this conjugacy class is 1 

g₀⁰ g₁⁰  

~~~~~~~~~~~~~~~~~~~~
Conjugacy Class number: 1
The size of this conjugacy class is 5 

g₀⁰ g₁¹  

g₀² g₁¹  

g₀⁴ g₁¹  

g₀¹ g₁¹  

g₀³ g₁¹  

~~~~~~~~~~~~~~~~~~~~
Conjugacy Class number: 2
The size of this conjugacy class is 2 

g₀¹ g₁⁰  

g₀⁴ g₁⁰  

~~~~~~~~~~~~~~~~~~~~
Conjugacy Class number: 3
The size of this conjugacy class is 2 

g₀² g₁⁰  

g₀³ g₁⁰  

~~~~~~~~~~~~~~~~~~~~


In [9]:
#NOTE: eventually when we want a complete state space we'll have to calculate the centralizer for
#      each element so that when we take the sum of elements in each conjugacy class we can calculate
#      the invariant polynomials properly

#choose representatives from each conjugacy class (just the first element in the list)
conjugacyRepresentatives = []
for conjugacyClass in groupConjugacyClasses:
    conjugacyRepresentatives.append(conjugacyClass[0])
    
#find the centralizer for each conjugacy class representative
centralizers = []
for index, element in enumerate(groupElements):
    print("\rWorking on centralizer:", index+1, "out of", len(groupElements), end = '\r', flush = True)
    centralizer = []
    for groupElement in groupElements:
        if (element * groupElement == groupElement * element):
            centralizer.append(groupElement)
    centralizers.append(centralizer)
    
#print the centralizer for each representative
#the numbers correspond to the conjugacy class of the representative they were calculated for
# print("\n~~~~~~~~~~~~~~~~~Printing the Centralizers~~~~~~~~~~~~~~~~~\n")
# for index, centralizer in enumerate(centralizers):
#     print("Centralizer number:" + str(index))
#     for element in centralizer:
#         print(element, "\n")
#     print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    

In [10]:
#get the index of every element in all centralizers within the group and save another copy
#of centralizers for the group product representation
centralizersSTR = []
for index, centralizer in enumerate(centralizers):
    print("\rWorking on centralizer:", index+1, "out of", len(groupElements), end = '\r', flush = True)
    centralizerSTR = []
    for element in centralizer:
        for index, tmpElement in enumerate(groupElements):
            if (tmpElement == element):
                centralizerSTR.append(groupElementsSTR[index])
    centralizersSTR.append(centralizerSTR)

#print the centralizer for each representative
#the numbers correspond to the conjugacy class of the representative they were calculated for
for index, centralizerSTR in enumerate(centralizersSTR):
    print("Centralizer number: " + str(index))
    print("The size of this centralizer is", len(centralizerSTR), "\n")
    for element in centralizerSTR:
        print(element, "\n")
    print("~~~~~~~~~~~~~~~~~~~~")

Centralizer number: 0r: 10 out of 108 out of 10
The size of this centralizer is 10 

g₀⁰ g₁⁰  

g₀⁰ g₁¹  

g₀¹ g₁⁰  

g₀¹ g₁¹  

g₀² g₁⁰  

g₀² g₁¹  

g₀³ g₁⁰  

g₀³ g₁¹  

g₀⁴ g₁⁰  

g₀⁴ g₁¹  

~~~~~~~~~~~~~~~~~~~~
Centralizer number: 1
The size of this centralizer is 2 

g₀⁰ g₁⁰  

g₀⁰ g₁¹  

~~~~~~~~~~~~~~~~~~~~
Centralizer number: 2
The size of this centralizer is 5 

g₀⁰ g₁⁰  

g₀¹ g₁⁰  

g₀² g₁⁰  

g₀³ g₁⁰  

g₀⁴ g₁⁰  

~~~~~~~~~~~~~~~~~~~~
Centralizer number: 3
The size of this centralizer is 2 

g₀⁰ g₁⁰  

g₀¹ g₁¹  

~~~~~~~~~~~~~~~~~~~~
Centralizer number: 4
The size of this centralizer is 5 

g₀⁰ g₁⁰  

g₀¹ g₁⁰  

g₀² g₁⁰  

g₀³ g₁⁰  

g₀⁴ g₁⁰  

~~~~~~~~~~~~~~~~~~~~
Centralizer number: 5
The size of this centralizer is 2 

g₀⁰ g₁⁰  

g₀² g₁¹  

~~~~~~~~~~~~~~~~~~~~
Centralizer number: 6
The size of this centralizer is 5 

g₀⁰ g₁⁰  

g₀¹ g₁⁰  

g₀² g₁⁰  

g₀³ g₁⁰  

g₀⁴ g₁⁰  

~~~~~~~~~~~~~~~~~~~~
Centralizer number: 7
The size of this centralizer is 2 

g₀⁰ g₁⁰  

g₀³ g₁¹  


In [13]:
#find all eigenvectors for each representative
repEigenvectors = []
for element in groupElements:
    repEigenvectors.append(element.eigenvectors_right())

#the eigenvectors are given by sage as (e,V,n)
#e is the eigenvalue
#V is the eigenvector
#n is the multiplicity

#run through each set of eigenvectors and find only the ones with eigenvalue of 1
fixG = []
for index, eigenvectors in enumerate(repEigenvectors):
    print("\rWorking on fix:", index+1, "out of", len(groupElements), end = '\r', flush = True)
    tmpEigenvectors = [] 
    for eigenvector in eigenvectors:
        radius = (eigenvector[0].real())^2 + (eigenvector[0].imag())^2
        angle = tan((eigenvector[0].imag())/(eigenvector[0].real()))
        realPart = eigenvector[0].real()
        #check that the eigenvalue on the complex plane is near the unit circle, the angle is within .5 degrees of zero
        #and that the real part is positive. All of this so that even with rounding error 
        #it can still detect 1 as an eigenvalue
        if ((.999 <= radius <= 1.001) and ((abs(angle)) < (pi/360)) and (realPart > 0)):
            tmpEigenvector = []
            for i in range(len(eigenvector[1][0])):
                if (eigenvector[1][0][i] != 0):
                    tmpEigenvector = eigenvector[1][0] / eigenvector[1][0][i]
                    break
            for i in range(len(eigenvector[1][0])):
                tmpEigenvector[i] = round(tmpEigenvector[i].real(), 6) + round(tmpEigenvector[i].imag(), 6)
            tmpEigenvectors.append(tmpEigenvector)
    fixG.append(tmpEigenvectors)          

#print the representative along with its fix locus
#the numbers represent the conjugacy class that the representative is from
print("\n\n~~~~~~~~~~~~~~~~~~~~~~Fix(G) for the representatives of the conjugacy classes~~~~~~~~~~~~~~~~~~~~~~")
for index, eigenvectors in enumerate(fixG):
    print("Fix G number:" + str(index))
    print("Element:")
    print(groupElements[index])
    print("Fix(g):")
    print(eigenvectors)
    print("")

Working on fix: 10 out of 10

~~~~~~~~~~~~~~~~~~~~~~Fix(G) for the representatives of the conjugacy classes~~~~~~~~~~~~~~~~~~~~~~
Fix G number:0
Element:
[1.0 0.0 0.0 0.0 0.0]
[0.0 1.0 0.0 0.0 0.0]
[0.0 0.0 1.0 0.0 0.0]
[0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 1.0]
Fix(g):
[(1.0, 0.0, 0.0, 0.0, 0.0), (0.0, 1.0, 0.0, 0.0, 0.0), (0.0, 0.0, 1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0, 0.0), (0.0, 0.0, 0.0, 0.0, 1.0)]

Fix G number:1
Element:
[0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 1.0 0.0 0.0]
[0.0 1.0 0.0 0.0 0.0]
[1.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 1.0]
Fix(g):
[(1.0, 0.0, 0.0, 1.0, 0.0), (0.0, 1.0, 1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0, 1.0)]

Fix G number:2
Element:
[0.0 1.0 0.0 0.0 0.0]
[0.0 0.0 1.0 0.0 0.0]
[0.0 0.0 0.0 1.0 0.0]
[0.0 0.0 0.0 0.0 1.0]
[1.0 0.0 0.0 0.0 0.0]
Fix(g):
[(1.0, 1.0, 1.0, 1.0, 1.0)]

Fix G number:3
Element:
[0.0 0.0 1.0 0.0 0.0]
[0.0 1.0 0.0 0.0 0.0]
[1.0 0.0 0.0 0.0 0.0]
[0.0 0.0 0.0 0.0 1.0]
[0.0 0.0 0.0 1.0 0.0]
Fix(g):
[(1.0, 0.0, 1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0, 1.0)

In [14]:
#get the index of every element in all fix locus representatives within the group and save another copy
#of fix locus representatives for the group product representation
elementSTR = []
for index, rep in enumerate(groupElements):
    print("\rWorking on fix:", index+1, "out of", len(groupElements), end = '\r', flush = True)
    for index, tmpElement in enumerate(groupElements):
        if (tmpElement == rep):
            elementSTR.append(groupElementsSTR[index])

#print the representative along with its fix locus
#the numbers represent the conjugacy class that the representative is from
print("\n\n~~~~~~~~~~~~~~~~~~~~~~Fix(G) for the representatives of the conjugacy classes~~~~~~~~~~~~~~~~~~~~~~")
for index, eigenvectors in enumerate(fixG):
    print("Fix G number:" + str(index))
    print("Element:")
    print(elementSTR[index])
    print("Fix(g):")
    print(eigenvectors)
    print("")

Working on fix: 10 out of 100 out of 10

~~~~~~~~~~~~~~~~~~~~~~Fix(G) for the representatives of the conjugacy classes~~~~~~~~~~~~~~~~~~~~~~
Fix G number:0
Element:
g₀⁰ g₁⁰ 
Fix(g):
[(1.0, 0.0, 0.0, 0.0, 0.0), (0.0, 1.0, 0.0, 0.0, 0.0), (0.0, 0.0, 1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0, 0.0), (0.0, 0.0, 0.0, 0.0, 1.0)]

Fix G number:1
Element:
g₀⁰ g₁¹ 
Fix(g):
[(1.0, 0.0, 0.0, 1.0, 0.0), (0.0, 1.0, 1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0, 1.0)]

Fix G number:2
Element:
g₀¹ g₁⁰ 
Fix(g):
[(1.0, 1.0, 1.0, 1.0, 1.0)]

Fix G number:3
Element:
g₀¹ g₁¹ 
Fix(g):
[(1.0, 0.0, 1.0, 0.0, 0.0), (0.0, 0.0, 0.0, 1.0, 1.0), (0.0, 1.0, 0.0, 0.0, 0.0)]

Fix G number:4
Element:
g₀² g₁⁰ 
Fix(g):
[(1.0, 1.0, 1.0, 1.0, 1.0)]

Fix G number:5
Element:
g₀² g₁¹ 
Fix(g):
[(1.0, 1.0, 0.0, 0.0, 0.0), (0.0, 0.0, 1.0, 0.0, 1.0), (0.0, 0.0, 0.0, 1.0, 0.0)]

Fix G number:6
Element:
g₀³ g₁⁰ 
Fix(g):
[(1.0, 1.0, 1.0, 1.0, 1.0)]

Fix G number:7
Element:
g₀³ g₁¹ 
Fix(g):
[(0.0, 1.0, 0.0, 0.0, 1.0), (0.0, 0.0, 1.0, 1.0, 0.0), (1.